In [22]:
import os
import tvm
from tvm.script import ir as I
from tvm.script import tir as T
from tvm import autotvm, auto_scheduler
from tvm.autotvm.tuner import XGBTuner, GATuner, RandomTuner, GridSearchTuner
from tvm import meta_schedule as ms
from tvm.ir import IRModule
from tvm import relax
from tvm import rpc
from tvm.contrib import utils, ndk
x_shape = 11008
w_w_x = 1376
w_s_x = 344
w_y = 4096
func_name = "main"
@I.ir_module
class ModuleSrc:
    @T.prim_func(private=False)
    # fused_fused_decode5_fused_NT_matmul4_add
    def main(lv17: T.Buffer((T.int64(1376), T.int64(4096)), "uint32"), lv18: T.Buffer((T.int64(344), T.int64(4096)), "float16"), p_lv16: T.handle, p_lv12: T.handle, p_output0: T.handle):
        T.func_attr({"tir.noalias": T.bool(True)})
        n = T.int64()
        lv16 = T.match_buffer(p_lv16, (T.int64(1), n, T.int64(11008)), "float16")
        lv12 = T.match_buffer(p_lv12, (T.int64(1), n, T.int64(4096)), "float16")
        p_output0_intermediate = T.match_buffer(p_output0, (T.int64(1), n, T.int64(4096)), "float16")
        # with T.block("root"):
        decode = T.alloc_buffer((T.int64(11008), T.int64(4096)), "float16")
        p_output0_intermediate_1 = T.alloc_buffer((T.int64(4096), T.int64(11008)), "float16")
        var_NT_matmul_intermediate = T.alloc_buffer((T.int64(1), n, T.int64(4096)), "float16")
        for i, j in T.grid(T.int64(11008), T.int64(4096)):
            with T.block("decode"):
                v_i, v_j = T.axis.remap("SS", [i, j])
                T.reads(lv17[v_i // T.int64(8), v_j], lv18[v_i // T.int64(32), v_j])
                T.writes(decode[v_i, v_j])
                decode[v_i, v_j] = (T.Cast("float16", T.bitwise_and(T.shift_right(lv17[v_i // T.int64(8), v_j], T.Cast("uint32", v_i % T.int64(8)) * T.uint32(4)), T.uint32(15))) - T.float16(7)) * lv18[v_i // T.int64(32), v_j]
        for ax0, ax1 in T.grid(T.int64(4096), T.int64(11008)):
            with T.block("T_transpose"):
                v_ax0, v_ax1 = T.axis.remap("SS", [ax0, ax1])
                T.reads(decode[v_ax1, v_ax0])
                T.writes(p_output0_intermediate_1[v_ax0, v_ax1])
                p_output0_intermediate_1[v_ax0, v_ax1] = decode[v_ax1, v_ax0]
        for i0, i1, i2, k in T.grid(T.int64(1), n, T.int64(4096), T.int64(11008)):
            with T.block("NT_matmul"):
                v_i0, v_i1, v_i2, v_k = T.axis.remap("SSSR", [i0, i1, i2, k])
                T.reads(lv16[v_i0, v_i1, v_k], p_output0_intermediate_1[v_i2, v_k])
                T.writes(var_NT_matmul_intermediate[v_i0, v_i1, v_i2])
                with T.init():
                    var_NT_matmul_intermediate[v_i0, v_i1, v_i2] = T.float16(0)
                var_NT_matmul_intermediate[v_i0, v_i1, v_i2] = var_NT_matmul_intermediate[v_i0, v_i1, v_i2] + lv16[v_i0, v_i1, v_k] * p_output0_intermediate_1[v_i2, v_k]
        for ax0, ax1, ax2 in T.grid(T.int64(1), n, T.int64(4096)):
            with T.block("T_add"):
                v_ax0, v_ax1, v_ax2 = T.axis.remap("SSS", [ax0, ax1, ax2])
                T.reads(lv12[v_ax0, v_ax1, v_ax2], var_NT_matmul_intermediate[v_ax0, v_ax1, v_ax2])
                T.writes(p_output0_intermediate[v_ax0, v_ax1, v_ax2])
                p_output0_intermediate[v_ax0, v_ax1, v_ax2] = lv12[v_ax0, v_ax1, v_ax2] + var_NT_matmul_intermediate[v_ax0, v_ax1, v_ax2]

@I.ir_module
class ModuleToManual:
    @T.prim_func(private=False)
    #fused_decode2_fused_NT_matmul3_add_after
    def main(
        lv50: T.Buffer((T.int64(1376), T.int64(4096)), "uint32"),
        lv51: T.Buffer((T.int64(344), T.int64(4096)), "float16"),
        p_lv5: T.handle,
        p_lv3: T.handle,
        p_output0: T.handle,
    ):
        T.func_attr({"tir.noalias": T.bool(True), "tir.is_scheduled": 1})
        n = T.int64()
        lv5 = T.match_buffer(p_lv5, (T.int64(1), n, T.int64(11008)), "float16")
        lv3 = T.match_buffer(p_lv3, (T.int64(1), n, T.int64(4096)), "float16")
        p_output0_intermediate = T.match_buffer(
            p_output0, (T.int64(1), n, T.int64(4096)), "float16"
        )
        # with T.block("root"):
        decode_local = T.alloc_buffer(
            (T.int64(11008), T.int64(4096)), "float16", scope="local"
        )
        lv50_local = T.alloc_buffer((T.int64(1376), T.int64(4096)), "uint32", scope="local")
        lv51_local = T.alloc_buffer((T.int64(344), T.int64(4096)), "float16", scope="local")
        lv5_pad_local = T.alloc_buffer(
            (T.int64(1), (n + T.int64(31)) // T.int64(32) * T.int64(32), T.int64(11008)),
            "float16",
            scope="local",
        )
        var_NT_matmul_intermediate_pad_local = T.alloc_buffer(
            (T.int64(1), (n + T.int64(31)) // T.int64(32) * T.int64(32), T.int64(4096)),
            "float16",
            scope="local",
        )
        # 任务划分:
        ### 一个thread处理 `processed_rows_per_thread`行 `vectorize_factor` 列(输出角度)
        ### 完整处理 `processed_rows_per_thread` 行输入需要: blockIdx.x * threadIdx.x 配合
        ### 完整处理 `n` 行输入需要: blockIdx.y * threadIdx.y 配合
        #### 分析: 根据`n`变化的只有 blockIdx.y, 说明 blockIdx.x * threadIdx.x * threadIdx.y 可以完整处理32行输入
        #  4 16 24 128 2
        BlockIdx_x = 32
        # n = 32
        # BlockIdx_y = (n+31)//32 * 32 # 这里32是假设输入为32的倍数, //32的32 = thready * 
        ThreadIdx_x = 32#16
        ThreadIdx_y = 16#8
        vectorize_factor = 4#8
        # processed_columns_per_thread = vectorize_factor# w_y / (BlockIdx_x * ThreadIdx_x) == vectorize_factor
        processed_rows_per_thread = 2# == 32 / threadIdx.y
        ### reduce拆分
        ## 当前各个threadIndex的含义：
        #   blockIdx.y  : Bb = (n + 31) // 32 * 32, 一次kernel调用用于处理 Bb 个32行输入
        #   blockIdx.x  : 32, 和 threadIdx.x, vectorize_factor 配合, 一个线程用于处理 4096 个元素(一行)
        #                   (= blockIdx.x * threadIdx.x * vectorize_factor), 此外单线程还会同时处理 processed_rows_per_thread 行
        #   threadIdx.x : 16, 和 blockIdx.x, vectorize_factor 配合, 一个线程用于处理 4096 个元素(一行)
        #                   (= blockIdx.x * threadIdx.x * vectorize_factor), 此外单线程还会同时处理 processed_rows_per_thread 行
        #   threadIdx.y : 8, 和 processed_rows_per_thread 配合用于处理 1个 32行输入
        ## 加一个threadIdx.z, 来处理拆分reduce的情况
        #   当前一列11008拆分情况(参数固定, 其中4和8受限于group_size和weight bits不可修改, 仅可拆344): 344 * 4 * 8
        #   rsplit_factor: [8, 4, 2] -> outter_loop: [43, 86, 172]
        # rsplit_factor = 8
        # r_outter_loop = 344 // rsplit_factor
        # assert 344 % rsplit_factor == 0, "344 is not divisible by rsplit_factor"

        for i0_i1_fused_0_i0_i1_fused_1_0_fused in T.thread_binding(
            (n + T.int64(31)) // T.int64(32), thread="blockIdx.y"
        ):
            for i2_0 in T.thread_binding(T.int64(BlockIdx_x), thread="blockIdx.x"):
                for i0_i1_fused_1_1 in T.thread_binding(T.int64(ThreadIdx_y), thread="threadIdx.y"):
                    for i2_1 in T.thread_binding(T.int64(ThreadIdx_x), thread="threadIdx.x"):
                        for i0_i1_fused_1_2_init in range(T.int64(processed_rows_per_thread)):
                            for i2_2_init in T.vectorized(T.int64(vectorize_factor)):
                                with T.block("NT_matmul_init"):
                                    v_i0 = T.axis.spatial(T.int64(1), T.int64(0))
                                    v_i1 = T.axis.spatial(
                                        (n + T.int64(31)) // T.int64(32) * T.int64(32),
                                        i0_i1_fused_0_i0_i1_fused_1_0_fused * T.int64(32)
                                        + i0_i1_fused_1_1 * processed_rows_per_thread
                                        + i0_i1_fused_1_2_init,
                                    )
                                    v_i2 = T.axis.spatial(
                                        T.int64(4096),
                                        i2_0 * (ThreadIdx_x * vectorize_factor) + i2_1 * vectorize_factor + i2_2_init,
                                    )
                                    T.reads()
                                    T.writes(
                                        var_NT_matmul_intermediate_pad_local[
                                            v_i0, v_i1, v_i2
                                        ]
                                    )
                                    var_NT_matmul_intermediate_pad_local[
                                        v_i0, v_i1, v_i2
                                    ] = T.float16(0)
                        for k_0 in range(T.int64(344)):
                            for ax0 in range(T.int64(1)):
                                for ax1 in T.vectorized(T.int64(vectorize_factor)):
                                    with T.block("lv51_local"):
                                        v0 = T.axis.spatial(T.int64(344), k_0 + ax0)
                                        v1 = T.axis.spatial(
                                            T.int64(4096),
                                            i2_0 * (ThreadIdx_x * vectorize_factor) + i2_1 * vectorize_factor + ax1,
                                        )
                                        T.reads(lv51[v0, v1])
                                        T.writes(lv51_local[v0, v1])
                                        lv51_local[v0, v1] = lv51[v0, v1]
                            for k_1 in range(T.int64(4)):
                                for ax0 in range(T.int64(1)):
                                    for ax1 in T.vectorized(vectorize_factor):
                                        with T.block("lv50_local"):
                                            v0 = T.axis.spatial(
                                                T.int64(1376), k_0 * T.int64(4) + k_1 + ax0
                                            )
                                            v1 = T.axis.spatial(
                                                T.int64(4096),
                                                i2_0 * (ThreadIdx_x * vectorize_factor)
                                                + i2_1 * vectorize_factor
                                                + ax1,
                                            )
                                            T.reads(lv50[v0, v1])
                                            T.writes(lv50_local[v0, v1])
                                            lv50_local[v0, v1] = lv50[v0, v1]
                                for k_2 in range(T.int64(8)):
                                    for ax0 in range(T.int64(1)):
                                        for ax1 in T.vectorized(vectorize_factor):
                                            with T.block("decode"):
                                                v_i = T.axis.spatial(
                                                    T.int64(11008),
                                                    k_0 * T.int64(32)
                                                    + k_1 * T.int64(8)
                                                    + k_2
                                                    + ax0,
                                                )
                                                v_j = T.axis.spatial(
                                                    T.int64(4096),
                                                    i2_0 * (ThreadIdx_x * vectorize_factor)
                                                    + i2_1 * vectorize_factor
                                                    + ax1,
                                                )
                                                T.reads(
                                                    lv50_local[v_i // T.int64(8), v_j],
                                                    lv51_local[v_i // T.int64(32), v_j],
                                                )
                                                T.writes(decode_local[v_i, v_j])
                                                decode_local[v_i, v_j] = (
                                                    T.Cast(
                                                        "float16",
                                                        T.bitwise_and(
                                                            T.shift_right(
                                                                lv50_local[
                                                                    v_i // T.int64(8), v_j
                                                                ],
                                                                T.Cast(
                                                                    "uint32",
                                                                    v_i % T.int64(8),
                                                                )
                                                                * T.uint32(4),
                                                            ),
                                                            T.uint32(15),
                                                        ),
                                                    )
                                                    - T.float16(7)
                                                ) * lv51_local[v_i // T.int64(32), v_j]
                                    for ax0, ax1 in T.grid(T.int64(1), processed_rows_per_thread):
                                        for ax2 in T.vectorized(T.int64(1)):
                                            with T.block("lv5_pad_local"):
                                                v0 = T.axis.spatial(T.int64(1), ax0)
                                                v1 = T.axis.spatial(
                                                    (n + T.int64(31))
                                                    // T.int64(32)
                                                    * T.int64(32),
                                                    i0_i1_fused_0_i0_i1_fused_1_0_fused
                                                    * T.int64(32)
                                                    + i0_i1_fused_1_1 * processed_rows_per_thread
                                                    + ax1,
                                                )
                                                v2 = T.axis.spatial(
                                                    T.int64(11008),
                                                    k_0 * T.int64(32)
                                                    + k_1 * T.int64(8)
                                                    + k_2
                                                    + ax2,
                                                )
                                                T.reads(lv5[v0, v1, v2])
                                                T.writes(lv5_pad_local[v0, v1, v2])
                                                lv5_pad_local[v0, v1, v2] = T.if_then_else(
                                                    v1 < n, lv5[v0, v1, v2], T.float16(0)
                                                )
                                    for i0_i1_fused_1_2 in range(processed_rows_per_thread):
                                        for i2_2 in T.vectorized(vectorize_factor):
                                            with T.block("NT_matmul_update"):
                                                v_i0 = T.axis.spatial(
                                                    T.int64(1), T.int64(0)
                                                )
                                                v_i1 = T.axis.spatial(
                                                    (n + T.int64(31))
                                                    // T.int64(32)
                                                    * T.int64(32),
                                                    i0_i1_fused_0_i0_i1_fused_1_0_fused
                                                    * T.int64(32)
                                                    + i0_i1_fused_1_1 * processed_rows_per_thread
                                                    + i0_i1_fused_1_2,
                                                )
                                                v_i2 = T.axis.spatial(
                                                    T.int64(4096),
                                                    i2_0 * (ThreadIdx_x * vectorize_factor)
                                                    + i2_1 * vectorize_factor
                                                    + i2_2,
                                                )
                                                v_k = T.axis.reduce(
                                                    T.int64(11008),
                                                    k_0 * T.int64(32)
                                                    + k_1 * T.int64(8)
                                                    + k_2,
                                                )
                                                T.reads(
                                                    var_NT_matmul_intermediate_pad_local[
                                                        v_i0, v_i1, v_i2
                                                    ],
                                                    lv5_pad_local[v_i0, v_i1, v_k],
                                                    decode_local[v_k, v_i2],
                                                )
                                                T.writes(
                                                    var_NT_matmul_intermediate_pad_local[
                                                        v_i0, v_i1, v_i2
                                                    ]
                                                )
                                                var_NT_matmul_intermediate_pad_local[
                                                    v_i0, v_i1, v_i2
                                                ] = (
                                                    var_NT_matmul_intermediate_pad_local[
                                                        v_i0, v_i1, v_i2
                                                    ]
                                                    + lv5_pad_local[v_i0, v_i1, v_k]
                                                    * decode_local[v_k, v_i2]
                                                )
                        for ax0, ax1 in T.grid(T.int64(1), processed_rows_per_thread):
                            for ax2 in T.vectorized(vectorize_factor):
                                with T.block("var_NT_matmul_intermediate_pad_local"):
                                    v0 = T.axis.spatial(T.int64(1), ax0)
                                    v1 = T.axis.spatial(
                                        (n + T.int64(31)) // T.int64(32) * T.int64(32),
                                        i0_i1_fused_0_i0_i1_fused_1_0_fused * T.int64(32)
                                        + i0_i1_fused_1_1 * processed_rows_per_thread
                                        + ax1,
                                    )
                                    v2 = T.axis.spatial(
                                        T.int64(4096),
                                        i2_0 * (ThreadIdx_x * vectorize_factor) + i2_1 * vectorize_factor + ax2,
                                    )
                                    T.reads(
                                        lv3[v0, v1, v2],
                                        var_NT_matmul_intermediate_pad_local[v0, v1, v2],
                                    )
                                    T.writes(p_output0_intermediate[v0, v1, v2])
                                    if v1 < n:
                                        p_output0_intermediate[v0, v1, v2] = (
                                            lv3[v0, v1, v2]
                                            + var_NT_matmul_intermediate_pad_local[
                                                v0, v1, v2
                                            ]
                                        )
sch_manual = tvm.tir.Schedule(ModuleToManual)
print(sch_manual.mod.script())
print("================================================")
rt_mod = tvm.build(sch_manual.mod, target="opencl")
print(rt_mod.imported_modules[0].get_source())

# from tvm.script import ir as I
# from tvm.script import tir as T

@I.ir_module
class Module:
    @T.prim_func
    def main(lv50: T.Buffer((T.int64(1376), T.int64(4096)), "uint32"), lv51: T.Buffer((T.int64(344), T.int64(4096)), "float16"), p_lv5: T.handle, p_lv3: T.handle, p_output0: T.handle):
        T.func_attr({"tir.is_scheduled": 1, "tir.noalias": T.bool(True)})
        n = T.int64()
        lv5 = T.match_buffer(p_lv5, (T.int64(1), n, T.int64(11008)), "float16")
        lv3 = T.match_buffer(p_lv3, (T.int64(1), n, T.int64(4096)), "float16")
        p_output0_intermediate = T.match_buffer(p_output0, (T.int64(1), n, T.int64(4096)), "float16")
        # with T.block("root"):
        decode_local = T.alloc_buffer((T.int64(11008), T.int64(4096)), "float16", scope="local")
        lv50_local = T.alloc_buffer((T.int64(1376), T.int64(4096)), "uint32", scope="local")
        lv51_local = T.alloc_buffer((T.int64(344), T.int64(4096)), "float16", scope="local")
        lv5_pad_local = T.alloc

In [13]:
# run and compare with cuda
import numpy as np
def _detect_local_cuda():
    dev = tvm.cuda()
    if not dev.exist:
        return None
    return tvm.target.Target(
        {
            "kind": "cuda",
            "max_shared_memory_per_block": dev.max_shared_memory_per_block,
            "max_threads_per_block": dev.max_threads_per_block,
            "thread_warp_size": dev.warp_size,
            "registers_per_block": 65536,
            "arch": "sm_" + tvm.cuda().compute_version.replace(".", ""),
        }
    )
# target = tvm.target.Target("cuda", host="llvm")
target = _detect_local_cuda()

print(target)
# 定义计算任务
dev = tvm.cuda(0)

num_flop = 1228406784
seq_len = 32
W_w_np = np.random.uniform(size=(w_w_x, w_y)).astype("uint32")
W_s_np = np.random.uniform(size=(w_s_x, w_y)).astype("float16")
Input_np = np.random.uniform(size=(1, seq_len, x_shape)).astype("float16")
Add_np = np.random.uniform(size=(1, seq_len, w_y)).astype("float16")
# W_w_np = np.ones((w_w_x, w_y), np.uint32) * 1#.astype("uint32")
# W_s_np = np.ones((w_s_x, w_y), np.float16) * 1#.astype("float16") * 2
# Input_np = np.ones((1, 1, x_shape), np.float16)#.astype("float16")
Output_nd = tvm.nd.array(np.zeros((1, seq_len, w_y), dtype="float16"), dev)
def numpy_caculate():
    test_rows = 2
    test_cols = 10
    output = np.zeros((1, test_rows, test_cols), dtype = np.float16)
    W_w_inv_np = np.transpose(W_w_np)
    W_s_inv_np = np.transpose(W_s_np)
    for row in range(test_rows):
        for i in range(test_cols):
            for r in range(x_shape):
                temp = Input_np[0][row][r] * np.float16((W_w_inv_np[i][r // 8] >> ((r % 8) * 4) & (15)) - np.float16(7.0)) * W_s_inv_np[i][r // 32]
                output[0][row][i] = output[0][row][i] + temp
            output[0][row][i] = output[0][row][i] + Add_np[0][row][i]
    print(output)
    output = np.zeros((1, test_rows, test_cols), dtype = np.float16)
    for row in range(test_rows):
        for i in range(test_cols):
            for r in range(x_shape):
                temp = Input_np[0][row][r] * np.float16((W_w_np[r // 8][i] >> ((r % 8) * 4) & (15)) - np.float16(7.0)) * W_s_np[r // 32][i]
                temp_output = output[0][row][i]
                output[0][row][i] = temp_output + temp
                # print(f"{temp_output} + {temp} = {output[0][0][i]}")
            output[0][row][i] = output[0][row][i] + Add_np[0][row][i]
    print(output)
numpy_caculate()
def print_npdata(np_data: np.ndarray) :
    print(np_data)
    print_num = 20
    d = np_data.flatten()
    p_size = print_num if d.size > print_num else d.size
    print(d[:p_size])

cuda -keys=cuda,gpu -arch=sm_61 -max_num_threads=1024 -max_shared_memory_per_block=49152 -max_threads_per_block=1024 -registers_per_block=65536 -thread_warp_size=32


[[[-13696. -13152. -13264. -13496. -13016. -13104. -12216. -13536.
   -13608. -13192.]
  [-13720. -13192. -13448. -13288. -13016. -13432. -12512. -13832.
   -13864. -13000.]]]
[[[-13696. -13152. -13264. -13496. -13016. -13104. -12216. -13536.
   -13608. -13192.]
  [-13720. -13192. -13448. -13288. -13016. -13432. -12512. -13832.
   -13864. -13000.]]]


In [14]:
# cuda未优化版本测试
sch = tvm.tir.Schedule(ModuleSrc)
with target:
    src_gpu_mod = tvm.tir.transform.DefaultGPUSchedule()(sch.mod) ##
rt_mod = tvm.build(src_gpu_mod, target="cuda")
W_w_nd = tvm.nd.array(W_w_np, dev)
W_s_nd = tvm.nd.array(W_s_np, dev)
Input_nd = tvm.nd.array(Input_np, dev)
Add_nd = tvm.nd.array(Add_np, dev)
Output_nd = tvm.nd.array(np.zeros((1, seq_len, w_y), dtype="float16"), dev)
evaluator = rt_mod.time_evaluator("main", dev, number=100)
print("manual_evaluator GEMV-Blocking: %f GFLOPS" % (num_flop / evaluator(W_w_nd, W_s_nd, Input_nd, Add_nd, Output_nd).mean / 1e9))
# print(Output_nd.numpy())
print_npdata(Output_nd.numpy())

manual_evaluator GEMV-Blocking: 1.541656 GFLOPS
[[[-13704. -13152. -13264. ... -12968. -12312. -13288.]
  [-13720. -13192. -13448. ... -13112. -12416. -13640.]
  [-13576. -13032. -13376. ... -13136. -12424. -13384.]
  ...
  [-13520. -13096. -13384. ... -12880. -12552. -13552.]
  [-13800. -13144. -13328. ... -12864. -12464. -13640.]
  [-13664. -13232. -13360. ... -12952. -12376. -13360.]]]
[-13704. -13152. -13264. -13504. -13016. -13128. -12216. -13544. -13600.
 -13192. -13160. -12784. -13136. -13392. -13624. -13736. -12800. -11776.
 -12904. -12656.]


In [27]:
os.environ["TVM_NDK_CC"]="/home/sensetime/Android/Sdk/ndk/25.2.9519653/toolchains/llvm/prebuilt/linux-x86_64/bin/aarch64-linux-android33-clang++"
target = tvm.target.Target("opencl -device=adreno", host="llvm -mtriple=aarch64-linux-gnu")
device_key="android"
rpc_host = "10.4.236.32"
rpc_port = 9190
comp_target = tvm.target.Target("opencl", host="llvm -mtriple=aarch64-linux-android")  # TODO: Only support arm64 for now

def test_opencl(mod: tvm.IRModule, name_hint: str):
    # mod = tvm.lower(sch_manual.mod)
    print("Build ...")
    android_rt_mod = tvm.build(mod, target="opencl", target_host="llvm -mtriple=aarch64-linux-android")
    # print(android_rt_mod.imported_modules[0].get_source())
    temp = utils.tempdir()
    path_dso_cl = temp.relpath("dev_lib_cl.so")
    android_rt_mod.export_library(path_dso_cl, ndk.create_shared)

    print("Run GPU(OpenCL Flavor) test ...")
    # Establish remote connection with target hardware

    tracker = rpc.connect_tracker(rpc_host, rpc_port)
    remote = tracker.request(device_key, priority=0, session_timeout=60)
    print("Connect to device done.")
    dev = remote.cl(0)
    remote.upload(path_dso_cl)
    f1 = remote.load_module("dev_lib_cl.so")

    W_w_nd = tvm.nd.array(W_w_np, dev)
    W_s_nd = tvm.nd.array(W_s_np, dev)
    Input_nd = tvm.nd.array(Input_np, dev)
    Add_nd = tvm.nd.array(Add_np, dev)
    Output_nd = tvm.nd.array(np.zeros((1, seq_len, w_y), dtype="float16"), dev)
    test_number=32
    time_f = f1.time_evaluator(f1.entry_name, dev, number=test_number)
    cost = time_f(W_w_nd, W_s_nd, Input_nd, Add_nd, Output_nd).mean
    print("evaluator[%s] GEMV-Blocking: %fms with loop %d" % (name_hint, cost * 1000, test_number))
    print("evaluator[%s] GEMV-Blocking: %fGFLOPS" % (name_hint, num_flop / cost / 1e9))

    print_npdata(Output_nd.numpy())
    # return Output_nd.numpy()
    return cost*1000 # unit: ms

In [24]:
# 未优化版本opencl测试
from tvm import dlight as dl
sch = tvm.tir.Schedule(ModuleSrc)
with target:
    # src_gpu_mod = tvm.tir.transform.DefaultGPUSchedule()(sch.mod) ##
    mod_deploy = dl.ApplyDefaultSchedule(  # pylint: disable=not-callable
        dl.gpu.Matmul(),
        dl.gpu.GEMV(),
        dl.gpu.Reduction(),
        dl.gpu.GeneralReduction(),
        dl.gpu.Fallback(),
    )(sch.mod)
src_output = test_opencl(mod_deploy, "source")
# print_npdata(src_output)


Build ...
Run GPU(OpenCL Flavor) test ...
Connect to device done.
evaluator[source] GEMV-Blocking: 191.681360ms with loop 32
evaluator[source] GEMV-Blocking: 6.408588GFLOPS
[[[-13696. -13152. -13264. ... -12952. -12328. -13280.]
  [-13720. -13200. -13448. ... -13112. -12416. -13640.]
  [-13576. -13032. -13360. ... -13136. -12424. -13400.]
  ...
  [-13520. -13096. -13384. ... -12864. -12544. -13552.]
  [-13800. -13152. -13328. ... -12864. -12464. -13632.]
  [-13672. -13232. -13360. ... -12968. -12384. -13360.]]]
[-13696. -13152. -13264. -13496. -13016. -13104. -12224. -13544. -13608.
 -13192. -13160. -12784. -13128. -13400. -13616. -13728. -12816. -11776.
 -12896. -12640.]


In [25]:
#优化版本opencl测试
# print(sch_manual.mod)
opt_output = test_opencl(sch_manual.mod, "opted")
# print_npdata(opt_output)
np.testing.assert_equal(opt_output, src_output)

Build ...
Run GPU(OpenCL Flavor) test ...
Connect to device done.
evaluator[opted] GEMV-Blocking: 11.407832ms with loop 32
evaluator[opted] GEMV-Blocking: 107.681002GFLOPS
[[[-13696. -13152. -13264. ... -12952. -12328. -13280.]
  [-13720. -13200. -13448. ... -13112. -12416. -13640.]
  [-13576. -13032. -13360. ... -13136. -12424. -13400.]
  ...
  [-13520. -13096. -13384. ... -12864. -12544. -13552.]
  [-13800. -13152. -13328. ... -12864. -12464. -13632.]
  [-13672. -13232. -13360. ... -12968. -12384. -13360.]]]
[-13696. -13152. -13264. -13496. -13016. -13104. -12224. -13544. -13608.
 -13192. -13160. -12784. -13128. -13400. -13616. -13728. -12816. -11776.
 -12896. -12640.]


In [28]:
# 自动搜索
# 以32为倍数先搜一波
# @TODO: 探索更低的倍数，以降低padding的额外性能损耗
def auto_tune(record_file: str):
    from typing import Union
    def search(vf: int, pr: int, bx: int, tx: int, ty: int):
        """search by workgroup

        Args:
            blockIdxX (_type_): blockIdx.x
            threadIdxX (_type_): threadIdx.x
            vectorize_output (_type_): 输出的vectorize参数, 决定单线程输出多少个结果
            vectorize_input (list, optional): 输入X拷贝到shared_memory时的vectorize参数, 一般为4或8

        Returns:
            _type_: _description_
        """
        @I.ir_module
        class ModuleToManual:
            @T.prim_func(private=False)
            # fused_decode1_fused_NT_matmul2_silu_after
            def main(
                lv50: T.Buffer((T.int64(1376), T.int64(4096)), "uint32"),
                lv51: T.Buffer((T.int64(344), T.int64(4096)), "float16"),
                p_lv5: T.handle,
                p_lv3: T.handle,
                p_output0: T.handle,
            ):
                T.func_attr({"tir.noalias": T.bool(True), "tir.is_scheduled": 1})
                n = T.int64()
                lv5 = T.match_buffer(p_lv5, (T.int64(1), n, T.int64(11008)), "float16")
                lv3 = T.match_buffer(p_lv3, (T.int64(1), n, T.int64(4096)), "float16")
                p_output0_intermediate = T.match_buffer(
                    p_output0, (T.int64(1), n, T.int64(4096)), "float16"
                )
                # with T.block("root"):
                decode_local = T.alloc_buffer(
                    (T.int64(11008), T.int64(4096)), "float16", scope="local"
                )
                lv50_local = T.alloc_buffer((T.int64(1376), T.int64(4096)), "uint32", scope="local")
                lv51_local = T.alloc_buffer((T.int64(344), T.int64(4096)), "float16", scope="local")
                lv5_pad_local = T.alloc_buffer(
                    (T.int64(1), (n + T.int64(31)) // T.int64(32) * T.int64(32), T.int64(11008)),
                    "float16",
                    scope="local",
                )
                var_NT_matmul_intermediate_pad_local = T.alloc_buffer(
                    (T.int64(1), (n + T.int64(31)) // T.int64(32) * T.int64(32), T.int64(4096)),
                    "float16",
                    scope="local",
                )

                # 任务划分:
                ### 一个thread处理 `processed_rows_per_thread`行 `vectorize_factor` 列(输出角度)
                ### 完整处理 `processed_rows_per_thread` 行输入需要: blockIdx.x * threadIdx.x 配合
                ### 完整处理 `n` 行输入需要: blockIdx.y * threadIdx.y 配合
                #### 分析: 根据`n`变化的只有 blockIdx.y, 说明 blockIdx.x * threadIdx.x * threadIdx.y 可以完整处理32行输入
                BlockIdx_x = bx#32
                # n = 32
                # BlockIdx_y = (n+31)//32 * 32 # 这里32是假设输入为32的倍数, //32的32 = thready * 
                ThreadIdx_x = tx#16 * 3
                ThreadIdx_y = ty#8
                vectorize_factor = vf#8
                # processed_columns_per_thread = vectorize_factor# w_y / (BlockIdx_x * ThreadIdx_x) == vectorize_factor
                processed_rows_per_thread = pr#4

                ## BlockIdx.y == [BlockIdx.x, ThreadIdx.x, ThraedIdx.y] 解决 seq_length为32的处理
                for i0_i1_fused_0_i0_i1_fused_1_0_fused in T.thread_binding(
                    (n + T.int64(31)) // T.int64(32), thread="blockIdx.y"
                ):
                    for i2_0 in T.thread_binding(T.int64(BlockIdx_x), thread="blockIdx.x"):
                        for i0_i1_fused_1_1 in T.thread_binding(T.int64(ThreadIdx_y), thread="threadIdx.y"):
                            for i2_1 in T.thread_binding(T.int64(ThreadIdx_x), thread="threadIdx.x"):
                                for i0_i1_fused_1_2_init in range(T.int64(processed_rows_per_thread)):
                                    for i2_2_init in T.vectorized(T.int64(vectorize_factor)):
                                        with T.block("NT_matmul_init"):
                                            v_i0 = T.axis.spatial(T.int64(1), T.int64(0))
                                            v_i1 = T.axis.spatial(
                                                (n + T.int64(31)) // T.int64(32) * T.int64(32),
                                                i0_i1_fused_0_i0_i1_fused_1_0_fused * T.int64(32)
                                                + i0_i1_fused_1_1 * processed_rows_per_thread
                                                + i0_i1_fused_1_2_init,
                                            )
                                            v_i2 = T.axis.spatial(
                                                T.int64(4096),
                                                i2_0 * (ThreadIdx_x * vectorize_factor) + i2_1 * vectorize_factor + i2_2_init,
                                            )
                                            T.reads()
                                            T.writes(
                                                var_NT_matmul_intermediate_pad_local[
                                                    v_i0, v_i1, v_i2
                                                ]
                                            )
                                            var_NT_matmul_intermediate_pad_local[
                                                v_i0, v_i1, v_i2
                                            ] = T.float16(0)
                                for k_0 in range(T.int64(344)):
                                    for ax0 in range(T.int64(1)):
                                        for ax1 in T.vectorized(T.int64(vectorize_factor)):
                                            with T.block("lv51_local"):
                                                v0 = T.axis.spatial(T.int64(344), k_0 + ax0)
                                                v1 = T.axis.spatial(
                                                    T.int64(4096),
                                                    i2_0 * (ThreadIdx_x * vectorize_factor) + i2_1 * vectorize_factor + ax1,
                                                )
                                                T.reads(lv51[v0, v1])
                                                T.writes(lv51_local[v0, v1])
                                                lv51_local[v0, v1] = lv51[v0, v1]
                                    for k_1 in range(T.int64(4)):
                                        for ax0 in range(T.int64(1)):
                                            for ax1 in T.vectorized(vectorize_factor):
                                                with T.block("lv50_local"):
                                                    v0 = T.axis.spatial(
                                                        T.int64(1376), k_0 * T.int64(4) + k_1 + ax0
                                                    )
                                                    v1 = T.axis.spatial(
                                                        T.int64(4096),
                                                        i2_0 * (ThreadIdx_x * vectorize_factor)
                                                        + i2_1 * vectorize_factor
                                                        + ax1,
                                                    )
                                                    T.reads(lv50[v0, v1])
                                                    T.writes(lv50_local[v0, v1])
                                                    lv50_local[v0, v1] = lv50[v0, v1]
                                        for k_2 in range(T.int64(8)):
                                            for ax0 in range(T.int64(1)):
                                                for ax1 in T.vectorized(vectorize_factor):
                                                    with T.block("decode"):
                                                        v_i = T.axis.spatial(
                                                            T.int64(11008),
                                                            k_0 * T.int64(32)
                                                            + k_1 * T.int64(8)
                                                            + k_2
                                                            + ax0,
                                                        )
                                                        v_j = T.axis.spatial(
                                                            T.int64(4096),
                                                            i2_0 * (ThreadIdx_x * vectorize_factor)
                                                            + i2_1 * vectorize_factor
                                                            + ax1,
                                                        )
                                                        T.reads(
                                                            lv50_local[v_i // T.int64(8), v_j],
                                                            lv51_local[v_i // T.int64(32), v_j],
                                                        )
                                                        T.writes(decode_local[v_i, v_j])
                                                        decode_local[v_i, v_j] = (
                                                            T.Cast(
                                                                "float16",
                                                                T.bitwise_and(
                                                                    T.shift_right(
                                                                        lv50_local[
                                                                            v_i // T.int64(8), v_j
                                                                        ],
                                                                        T.Cast(
                                                                            "uint32",
                                                                            v_i % T.int64(8),
                                                                        )
                                                                        * T.uint32(4),
                                                                    ),
                                                                    T.uint32(15),
                                                                ),
                                                            )
                                                            - T.float16(7)
                                                        ) * lv51_local[v_i // T.int64(32), v_j]
                                            for ax0, ax1 in T.grid(T.int64(1), processed_rows_per_thread):
                                                for ax2 in T.vectorized(T.int64(1)):
                                                    with T.block("lv5_pad_local"):
                                                        v0 = T.axis.spatial(T.int64(1), ax0)
                                                        v1 = T.axis.spatial(
                                                            (n + T.int64(31))
                                                            // T.int64(32)
                                                            * T.int64(32),
                                                            i0_i1_fused_0_i0_i1_fused_1_0_fused
                                                            * T.int64(32)
                                                            + i0_i1_fused_1_1 * processed_rows_per_thread
                                                            + ax1,
                                                        )
                                                        v2 = T.axis.spatial(
                                                            T.int64(11008),
                                                            k_0 * T.int64(32)
                                                            + k_1 * T.int64(8)
                                                            + k_2
                                                            + ax2,
                                                        )
                                                        T.reads(lv5[v0, v1, v2])
                                                        T.writes(lv5_pad_local[v0, v1, v2])
                                                        lv5_pad_local[v0, v1, v2] = T.if_then_else(
                                                            v1 < n, lv5[v0, v1, v2], T.float16(0)
                                                        )
                                            for i0_i1_fused_1_2 in range(processed_rows_per_thread):
                                                for i2_2 in T.vectorized(vectorize_factor):
                                                    with T.block("NT_matmul_update"):
                                                        v_i0 = T.axis.spatial(
                                                            T.int64(1), T.int64(0)
                                                        )
                                                        v_i1 = T.axis.spatial(
                                                            (n + T.int64(31))
                                                            // T.int64(32)
                                                            * T.int64(32),
                                                            i0_i1_fused_0_i0_i1_fused_1_0_fused
                                                            * T.int64(32)
                                                            + i0_i1_fused_1_1 * processed_rows_per_thread
                                                            + i0_i1_fused_1_2,
                                                        )
                                                        v_i2 = T.axis.spatial(
                                                            T.int64(4096),
                                                            i2_0 * (ThreadIdx_x * vectorize_factor)
                                                            + i2_1 * vectorize_factor
                                                            + i2_2,
                                                        )
                                                        v_k = T.axis.reduce(
                                                            T.int64(11008),
                                                            k_0 * T.int64(32)
                                                            + k_1 * T.int64(8)
                                                            + k_2,
                                                        )
                                                        T.reads(
                                                            var_NT_matmul_intermediate_pad_local[
                                                                v_i0, v_i1, v_i2
                                                            ],
                                                            lv5_pad_local[v_i0, v_i1, v_k],
                                                            decode_local[v_k, v_i2],
                                                        )
                                                        T.writes(
                                                            var_NT_matmul_intermediate_pad_local[
                                                                v_i0, v_i1, v_i2
                                                            ]
                                                        )
                                                        var_NT_matmul_intermediate_pad_local[
                                                            v_i0, v_i1, v_i2
                                                        ] = (
                                                            var_NT_matmul_intermediate_pad_local[
                                                                v_i0, v_i1, v_i2
                                                            ]
                                                            + lv5_pad_local[v_i0, v_i1, v_k]
                                                            * decode_local[v_k, v_i2]
                                                        )
                                for ax0, ax1 in T.grid(T.int64(1), processed_rows_per_thread):
                                    for ax2 in T.vectorized(vectorize_factor):
                                        with T.block("var_NT_matmul_intermediate_pad_local"):
                                            v0 = T.axis.spatial(T.int64(1), ax0)
                                            v1 = T.axis.spatial(
                                                (n + T.int64(31)) // T.int64(32) * T.int64(32),
                                                i0_i1_fused_0_i0_i1_fused_1_0_fused * T.int64(32)
                                                + i0_i1_fused_1_1 * processed_rows_per_thread
                                                + ax1,
                                            )
                                            v2 = T.axis.spatial(
                                                T.int64(4096),
                                                i2_0 * (ThreadIdx_x * vectorize_factor) + i2_1 * vectorize_factor + ax2,
                                            )
                                            T.reads(
                                                lv3[v0, v1, v2],
                                                var_NT_matmul_intermediate_pad_local[v0, v1, v2],
                                            )
                                            T.writes(p_output0_intermediate[v0, v1, v2])
                                            if v1 < n:
                                                p_output0_intermediate[v0, v1, v2] = (
                                                    lv3[v0, v1, v2]
                                                    + var_NT_matmul_intermediate_pad_local[
                                                        v0, v1, v2
                                                    ]
                                                )
        return tvm.tir.Schedule(ModuleToManual).mod

    BlockIdx_x = [None] # 32
    # n = 32
    # BlockIdx_y = (n+31)//32 * 32 # 这里32是假设输入为32的倍数, //32的32 = thready * 
    ThreadIdx_x = [16, 32, 48, 64, 96, 128, 192, 256, 384, 512]#16 * 3
    # ThreadIdx_y = 8 # = 32 / processed_rows_per_thread
    vectorize_factor = [2, 4, 8]# 8
    # processed_columns_per_thread = vectorize_factor# w_y / (BlockIdx_x * ThreadIdx_x) == vectorize_factor
    processed_rows_per_thread = [1, 2, 4, 8, 16]#4
    task_index = 0
    total_task_num = len(vectorize_factor)*len(BlockIdx_x)*len(ThreadIdx_x)*len(processed_rows_per_thread)
    records = {}
    print(f"Total tasks: {total_task_num}")
    # try:
    vectorize_factor_r = vectorize_factor[::-1]
    processed_rows_per_thread_r = processed_rows_per_thread[::-1]
    ThreadIdx_x_r = ThreadIdx_x[::-1]
    # table
    write_interval = 5
    from prettytable import PrettyTable
    table = PrettyTable()
    table.field_names = ["vectorize_factor", "processed_rows_per_thread", "blockIdx.x", "threadIdx.x", "threadIdx.y", "cost(ms)"]
    for vf in vectorize_factor_r:
        for pr in processed_rows_per_thread_r:
            for tx in ThreadIdx_x_r:
                task_index = task_index + 1
                import math
                bx = math.ceil(w_y /(vf * tx))
                ty = math.ceil(32//pr)
                if tx * vf >= w_y or tx*ty > 1024 or 32 % pr != 0: # w_y为输出列数, 工作组和vectorize相乘不能大于该数字
                    print(f"search record [{task_index}/{total_task_num}]: skip {vf} {pr} {bx} {tx} {ty}")
                    continue
                if w_y % (vf * tx) != 0:
                    print(f"search record [{task_index}/{total_task_num}]: skip because bx isn't divisible {vf} {pr} {bx} {tx} {ty}")
                    continue
                print(f"search record [{task_index}/{total_task_num}]: start run {vf} {pr} {bx} {tx} {ty}")
                # vf: int, pr: int, bx: int, tx: int, ty: int):
                mod_deploy = search(vf, pr, bx, tx, ty)
                cost = test_opencl(mod_deploy, "search")
                print("=====")
                records[(vf, pr, bx, tx, ty)] = cost
                table.add_row([vf, pr, bx, tx, ty, cost])
                if task_index % write_interval == 0:
                    with open(record_file, 'wt') as f:
                        f.write(table.get_csv_string())
    # except Exception as e:
    #     print(f"error occured: {e}")
    ### write file
    # from prettytable import PrettyTable
    # table = PrettyTable()
    # table.field_names = ["vectorize_factor", "processed_rows_per_thread", "blockIdx.x", "threadIdx.x", "threadIdx.y", "cost(ms)"]
    # for config, cost in records.items():
    #     table.add_row([config[0], config[1], config[2], config[3], config[4], cost])
    #     print(f"{config}: {cost}ms")
    print("================================")
    print(table)
    with open(record_file, 'wt') as f:
        f.write(table.get_csv_string())
    
    # record_sorted = sorted(record.items(), key=lambda x: x[1][0], reverse=True)
auto_tune("./manual_tune/down_n_tune_record_1.csv")

Total tasks: 150
search record [1/150]: skip 8 16 1 512 2
search record [2/150]: skip because bx isn't divisible 8 16 2 384 2
search record [3/150]: start run 8 16 2 256 2
Build ...
Run GPU(OpenCL Flavor) test ...
Connect to device done.
evaluator[search] GEMV-Blocking: 184.540640ms with loop 32
evaluator[search] GEMV-Blocking: 6.656565GFLOPS
[[[-13696. -13152. -13264. ... -12952. -12328. -13280.]
  [-13720. -13200. -13448. ... -13112. -12416. -13640.]
  [-13576. -13032. -13360. ... -13136. -12424. -13400.]
  ...
  [-13520. -13096. -13384. ... -12864. -12544. -13552.]
  [-13800. -13152. -13328. ... -12864. -12464. -13632.]
  [-13672. -13232. -13360. ... -12968. -12384. -13360.]]]
[-13696. -13152. -13264. -13496. -13016. -13104. -12224. -13544. -13608.
 -13192. -13160. -12784. -13128. -13400. -13616. -13728. -12816. -11776.
 -12896. -12640.]
=====
search record [4/150]: skip because bx isn't divisible 8 16 3 192 2
search record [5/150]: start run 8 16 4 128 2
Build ...
Run GPU(OpenCL Fl

In [ ]:
import numpy as np
target = tvm.target.Target("opencl -device=adreno", host="llvm -mtriple=aarch64-linux-gnu")
device_key="android"
rpc_host = "10.158.176.30"
rpc_port = 5001
# remote = autotvm.measure.request_remote(device_key, "10.158.176.30", 5001, timeout=10000)
# dev = remote.device(str(target), 0)

# num_flop = 1228406784
# W_np = np.random.uniform(size=(512, vocab_size)).astype("uint32")
# S_np = np.random.uniform(size=(128, vocab_size)).astype("float16")
# Input_np = np.random.uniform(size=(1, 1, 4096)).astype("float16")
# # Output_np = np.random.uniform(size=(1, 1, 4096)).astype("float16")
# W_nd = tvm.nd.array(W_np, dev)
# S_nd = tvm.nd.array(S_np, dev)
# Input_nd = tvm.nd.array(Input_np, dev)
# Output_nd = tvm.nd.array(np.zeros((1, 1, vocab_size), dtype="float32"), dev)

In [ ]:
rpc_config = ms.runner.RPCConfig(tracker_host=rpc_host, tracker_port=rpc_port, tracker_key = device_key)
runner= ms.runner.RPCRunner(rpc_config)
# ms.builder.LocalBuilder()
sch = tvm.tir.Schedule(ModuleSrc)
database = ms.tune_tir(
    mod=ModuleSrc,
    target=target,
    max_trials_global=64,
    num_trials_per_iter=64,
    work_dir="./tune_first",
    cost_model="xgb",
    runner = runner
)
print(len(database))
sch1 = ms.tir_integration.compile_tir(database, sch.mod, target)
print(type(sch1))

In [ ]:
from tvm.script import relax as R
@I.ir_module
class Module:
    @R.function
    def main(A: R.Tensor((3, 4), dtype="float32"), B: R.Tensor((4, 5), dtype="float32")):
        with R.dataflow():
            lv: R.Tensor((3, 5), dtype="float32") = R.matmul(A, B)
            gv: R.Tensor((3, 5), dtype="float32") = lv
            R.output(gv)
        return gv

In [ ]:
## auto_scheduler test
from tvm import auto_scheduler
import numpy as np
a_np = np.random.rand(3, 4).astype("float32")
b_np = np.random.rand(4, 5).astype("float32")
a_nd = tvm.runtime.NDArray(a_np)
b_nd = tvm.runtime.NDArray(b_np)
sch = tvm.tir.Schedule(Module)

params = {"A": a_np, "B": b_np}
## 报错，这里只支持relay
# tasks = auto_scheduler.extract_tasks(sch.mod, params, target=target)
tasks = ms.relax_integration.extract_tasks(sch.mod, target=target, params=params)
print(len(tasks))

In [ ]:

from mod_deploy import Module as ModuleAll
params_all = {}
tasks_all = auto_scheduler.extract_tasks(ModuleAll, params_all, target=target)
print(len(tasks_all))

In [ ]:
import numpy as np
log_file = "tune.json"
def _detect_local_cuda():
    dev = tvm.cuda()
    if not dev.exist:
        return None
    return tvm.target.Target(
        {
            "kind": "cuda",
            "max_shared_memory_per_block": dev.max_shared_memory_per_block,
            "max_threads_per_block": dev.max_threads_per_block,
            "thread_warp_size": dev.warp_size,
            "registers_per_block": 65536,
            "arch": "sm_" + tvm.cuda().compute_version.replace(".", ""),
        }
    )
# target = tvm.target.Target("cuda", host="llvm")
target = _detect_local_cuda()

print(target)
# 定义计算任务
dev = tvm.cuda(0)

num_flop = 1228406784
W_np = np.random.uniform(size=(512, vocab_size)).astype("uint32")
S_np = np.random.uniform(size=(128, vocab_size)).astype("float16")
Input_np = np.random.uniform(size=(1, 1, 4096)).astype("float16")
# Output_np = np.random.uniform(size=(1, 1, 4096)).astype("float16")
W_nd = tvm.nd.array(W_np, dev)
S_nd = tvm.nd.array(S_np, dev)
Input_nd = tvm.nd.array(Input_np, dev)
Output_nd = tvm.nd.array(np.zeros((1, 1, vocab_size), dtype="float32"), dev)
sch = tvm.tir.Schedule(ModuleSrc)
new_mod = sch.mod


In [ ]:
# task = auto_scheduler.SearchTask(func=sch.mod['fused_fused_decode11_fused_matmul5_cast2'], args=sch.mod['fused_fused_decode11_fused_matmul5_cast2'].params, target=target)

# tune_option = auto_scheduler.TuningOptions(
#     num_measure_trials=10,
#     measure_callbacks=[auto_scheduler.RecordToFile(log_file)],
#     verbose=2,
# )


database = ms.tune_tir(
    mod=new_mod,
    target=target,
    max_trials_global=64,
    num_trials_per_iter=64,
    work_dir="./tune_45593_1",
    cost_model="xgb"
)
print(len(database))
sch1 = ms.tir_integration.compile_tir(database, new_mod, target)
print(type(sch1))

In [ ]:
# print(sch1.trace)
# print(sch1.mod.script())
rt_mod = tvm.build(sch1.mod, target="cuda")

evaluator = rt_mod.time_evaluator("main", dev, number=100)

print("evaluator GEMV-Blocking: %f GFLOPS" % (1228406784 / evaluator(W_nd, S_nd, Input_nd, Output_nd).mean / 1e9))




In [ ]:

record_database = ms.Database.create(kind='json', work_dir='./tune_45593_1')


In [ ]:
record_sch = ms.tir_integration.compile_tir(record_database, new_mod, target)

record_rt_mod = tvm.build(record_sch.mod, target="cuda")

record_evaluator = record_rt_mod.time_evaluator("main", dev, number=20)

print("evaluator GEMV-Blocking: %f GFLOPS" % (num_flop / record_evaluator(W_nd, S_nd, Input_nd, Output_nd).mean / 1e9))
print(record_sch.trace)
print(record_sch.mod.script())

In [ ]:
from typing import TYPE_CHECKING, Dict, List, Optional, Union, Callable
from tvm import runtime
if TYPE_CHECKING:
    import numpy as np  # type: ignore
    from tvm.ir import IRModule
    from tvm.meta_schedule.runner import EvaluatorConfig, RPCConfig
    from tvm.runtime import Device, Module, NDArray
    from tvm.target import Target
    from tvm.runtime.vm import Executable


def f_measurement(
    rt_mod: runtime.Module, device: runtime.ndarray.Device, input_data: Dict[str, runtime.NDArray]
):
    vm = relax.VirtualMachine(rt_mod, device=device)
    vm.save_function("main", "measure_func", **input_data, include_return=False)
    evaluator = vm.time_evaluator(
        func_name="measure_func",
        dev=device,
        repeat=100,
        number=1,
        min_repeat_ms=500,
    )
    return evaluator()

def run_module_via_rpc(
    rpc_config: "RPCConfig",
    lib: Union["Module", "Executable"],
    dev_type: str,
    args: Union[Dict[int, "np.ndarray"], Dict[str, "np.ndarray"]],
    continuation: Callable,
    backend: Optional[str] = "graph",
):
    """Execute a tvm.runtime.Module on RPC remote"""
    # pylint: disable=import-outside-toplevel
    import os
    import tempfile

    from tvm.contrib.tar import tar
    from tvm.runtime import ndarray

    # pylint: enable=import-outside-toplevel

    with tempfile.TemporaryDirectory() as tmp_dir:
        # filename = os.path.join(tmp_dir, "tvm_tmp_mod." + tar.output_format)
        filename = os.path.join(tmp_dir, "tvm_tmp_mod." + "so")
        if backend == "vm":
            code, lib = lib.save(filename, fmt="so")
        from tvm.contrib import ndk
        lib.export_library(filename, ndk.create_shared)
        session = rpc_config.connect_server()
        print(type(session._sess))
        session.upload(filename)
        _, filename = os.path.split(filename)
        rt_mod = session.load_module(filename)
        
        if backend == "vm":
            rt_mod = session.get_function("runtime.Load_Executable")(code, rt_mod)
            # rt_mod = session.get_function("runtime.module.loadfile_relax.Executable")(filename)
        dev = session.device(dev_type=dev_type, dev_id=0)
        # print(dev)
        # create the remote runtime module
        print(rt_mod)
        print(rt_mod['main'])
        from tvm.contrib import graph_executor as runtime
        module = runtime.GraphModule(rt_mod["main"](dev))
        print(module)
        for k, v in args.items():
            module.set_input(k, tvm.nd.array(v))
        return module.run()
        # nd_args = {k: ndarray.array(v, dev) for k, v in args.items()}
        nd_args = {k: ndarray.empty(v.shape, v.dtype, dev) for k, v in args.items()}
        return continuation(rt_mod, dev, nd_args)